In [ ]:
from bs4 import BeautifulSoup
import re
import pymysql
from urllib.request import urlopen

conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='medici#0', db='mysql', charset='utf8')
cur = conn.cursor()
cur.execute("USE wikipedia")

def pageScraped(url):
    cur.execute("SELECT * FROM pages WHERE url = %s", (url))
    if cur.rowcount == 0:
        return False
    page = cur.fetchone()
    
    cur.execute("SELECT * FROM links WHERE fromPageId = %s", (int(page[0])))
    if cur.rowcount == 0:
        return False
    return True

def insertPageIfNotExists(url):
    cur.execute("SELECT * FROM pages WHERE url = %s", (url))
    if cur.rowcount == 0:
        cur.execute("INSERT INTO pages (url) VALUES (%s)", (url))
        conn.commit()
        return cur.lastrowid
    else:
        return cur.fetchone()[0]
    
def insertLink(fromPageId, toPageId):
    cur.execute(
        "SELECT * FROM links WHERE fromPageId = %s AND toPageId = %s", (int(fromPageId), int(toPageId))
    )
    if cur.rowcount == 0:
        cur.execute(
            "INSERT INTO links (fromPageId, toPageId) VALUES (%s, %s)", (int(fromPageId), int(toPageId))
        )
        conn.commit()

def getLinks(pageUrl, recursionLevel):
    global pages
    
    if recursionLevel > 4:
        return
    
    pageId = insertPageIfNotExists(pageUrl)
    html = urlopen("http://en.wikipedia.org"+pageUrl)
    bsObj = BeautifulSoup(html, "html.parser")
    
    for link in bsObj.findAll("a", href=re.compile("^(/wiki/)((?!:).)*$")):
        insertLink(pageId, insertPageIfNotExists(link.attrs['href']))
        if not pageScraped(link.attrs['href']):
            newPage = link.attrs['href']
            print(newPage)
            getLinks(newPage, recursionLevel+1)
        else:
            print("Skipping: "+str(link.attrs['href'])+" found on "+pageUrl)

getLinks("/wiki/Kevin_Bacon", 0)
cur.close()
conn.close()

/wiki/Kevin_Bacon_(disambiguation)
Skipping: /wiki/Kevin_Bacon found on /wiki/Kevin_Bacon_(disambiguation)
/wiki/Kevin_Bacon_(producer)
/wiki/Rotherham
/wiki/Rotherham_(disambiguation)
Skipping: /wiki/Rotherham found on /wiki/Rotherham_(disambiguation)
/wiki/Rotherham_(UK_Parliament_constituency)
/wiki/Metropolitan_Borough_of_Rotherham
/wiki/Rotherham,_New_Zealand
/wiki/Alan_Rotherham
/wiki/Arthur_Rotherham
/wiki/Gerard_Rotherham
/wiki/Hugh_Rotherham
/wiki/Joseph_Bryant_Rotherham
/wiki/Roland_Rotherham
/wiki/Thomas_Rotherham
/wiki/Edward_Rotheram
/wiki/Steve_Rotheram
/wiki/HMS_Rotherham_(H09)
/wiki/Baron_Rotherham
/wiki/Rotherham_United_F.C.
/wiki/Rotherham_child_sexual_exploitation_scandal
Skipping: /wiki/Rotherham_(disambiguation) found on /wiki/Rotherham_(disambiguation)
Skipping: /wiki/Rotherham_(disambiguation) found on /wiki/Rotherham_(disambiguation)
/wiki/Main_Page
/wiki/Main_Page
/wiki/Rotherham_Minster
/wiki/Ordnance_Survey_National_Grid
/wiki/Christian_denomination
/wiki/Chu